In [1]:
# As documented in the NRPy+ tutorial module
#   Tutorial_SEOBNR_Derivative_Routine.ipynb,
#   this module computes partial derivatives
#   of the SEOBNRv3 Hamiltonian with respect
#   to 12 dynamic variables

# Authors: Zachariah B. Etienne & Tyler Knowles
#         zachetie **at** gmail **dot* com

# Step 1.a: import all needed modules from Python/NRPy+:
import sympy as sp                # SymPy: The Python computer algebra package upon which NRPy+ depends
import sys, os                    # Standard Python modules for multiplatform OS-level functions

nrpy_dir_path = os.path.join("..") # TYLERK: Remove these three lines when moving to main NRPy+ directory
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

from outputC import superfast_uniq, lhrh      # Remove duplicate entries from a Python array; store left- and right-
                                              #   hand sides of mathematical expressions

# Step 1.b: Check for a sufficiently new version of SymPy (for validation)
# Ignore the rc's and b's for release candidates & betas.
sympy_version = sp.__version__.replace('rc', '...').replace('b', '...')
sympy_version_decimal = float(int(sympy_version.split(".")[0]) + int(sympy_version.split(".")[1])/10.0)
if sympy_version_decimal < 1.2:
    print('Error: NRPy+ does not support SymPy < 1.2')
    sys.exit(1)

# Step 1.c: Name of the directory containing the input file
inputdir = "SEOBNR"

# Supporting function to simplify derivative expressions by removing terms equal to 0
def simplify_deriv(lhss_deriv,rhss_deriv):
    # Copy expressions into another array
    lhss_deriv_simp = []
    rhss_deriv_simp = []
    for i in range(len(rhss_deriv)):
        lhss_deriv_simp.append(lhss_deriv[i])
        rhss_deriv_simp.append(rhss_deriv[i])
    # If a right-hand side is 0, substitute value 0 for the corresponding left-hand side in later terms
    for i in range(len(rhss_deriv_simp)):
        if rhss_deriv_simp[i] == 0:
            for j in range(i+1,len(rhss_deriv_simp)):
                for var in rhss_deriv_simp[j].free_symbols:
                    if str(var) == str(lhss_deriv_simp[i]):
                        rhss_deriv_simp[j] = rhss_deriv_simp[j].subs(var,0)
    zero_elements_to_remove = []
    # Create array of indices for expressions that are zero
    for i in range(len(rhss_deriv_simp)):
        if rhss_deriv_simp[i] == sp.sympify(0):
            zero_elements_to_remove.append(i)

    # When removing terms that are zero, we need to take into account their new index (after each removal)
    count = 0
    for i in range(len(zero_elements_to_remove)):
        del lhss_deriv_simp[zero_elements_to_remove[i]+count]
        del rhss_deriv_simp[zero_elements_to_remove[i]+count]
        count -= 1
    return lhss_deriv_simp,rhss_deriv_simp

# Supporing function to convert a generic partial derivative into a partial derivative with respect to a specific variable
def deriv_onevar(lhss_deriv,rhss_deriv,variable_list,index):
    # Denote each variable with prm
    variableprm_list = []
    for variable in variable_list:
        variableprm_list.append(str(variable)+"Prm")

    # Copy expressions into another array
    lhss_deriv_new = []
    rhss_deriv_new = []
    for i in range(len(rhss_deriv)):
        lhss_deriv_new.append(lhss_deriv[i])
        rhss_deriv_new.append(rhss_deriv[i])
    # For each free symbol's derivative, replace it with:
    #   1, if we are differentiating with respect to the variable, or
    #   0, if we are note differentiating with respect to that variable
    for i in range(len(rhss_deriv_new)):
        for var in variableprm_list:
            if variableprm_list.index(str(var))==index:
            #if var==(variable+"prm"):
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,1)
            else:
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,0)
    # Simplify derivative expressions again
    lhss_deriv_simp,rhss_deriv_simp = simplify_deriv(lhss_deriv_new,rhss_deriv_new)
    return lhss_deriv_simp,rhss_deriv_simp


# Step 2.a: Read in expressions as a (single) string
with open(os.path.join(inputdir,'second_derivatives_input.txt'), 'r') as file:
    expressions_as_lines = file.readlines()

# Step 2.b: Create and populate the "lr" array, which separates each line into left- and right-hand sides
#   Each entry is a string of the form lhrh(lhs='',rhs='')
lr = []

for i in range(len(expressions_as_lines)):
    # Ignore lines with 2 or fewer characters and those starting with #
    if len(expressions_as_lines[i]) > 2 and expressions_as_lines[i][0] != "#":
        # Split each line by its equals sign
        split_line = expressions_as_lines[i].split("=")
        # Append the line to "lr", removing spaces, "sp." prefixes, and replacing Lambda->Lamb
        #   (Lambda is a protected keyword):
        lr.append(lhrh(lhs=split_line[0].replace(" ","").replace("Lambda","Lamb"),
                           rhs=split_line[1].replace(" ","").replace("sp.","").replace("Lambda","Lamb")))

# Step 2.c: Separate and sympify right- and left-hand sides into separate arrays
lhss = []
rhss = []
for i in range(len(lr)):
    lhss.append(sp.sympify(lr[i].lhs))
    rhss.append(sp.sympify(lr[i].rhs))

# Step 3.a: Create `input_constants` array and populate with SymPy symbols
m1,m2,tortoise,eta,KK,k0,k1,EMgamma,d1v2,dheffSSv2 = sp.symbols('m1 m2 tortoise eta KK k0 k1 EMgamma d1v2 dheffSSv2',
                                                                    real=True)
s1x,s1y,s1z,s2x,s2y,s2z = sp.symbols('s1x s1y s1z s2x s2y s2z', real=True)
input_constants = [m1,m2,tortoise,eta,KK,k0,k1,EMgamma,d1v2,dheffSSv2,s1x,s1y,s1z,s2x,s2y,s2z]

# Step 3.b: Create `dynamic_variables` array and populate with SymPy symbols
x,y,z,px,py,pz = sp.symbols('x y z px py pz', real=True)
dynamic_variables = [x,y,z,px,py,pz]

# Step 4.a: Prepare array of "free symbols" in the right-hand side expressions
full_symbol_list_with_dups = []
for i in range(len(lr)):
    for variable in rhss[i].free_symbols:
        full_symbol_list_with_dups.append(variable)

# Step 4.b: Remove duplicate free symbols
full_symbol_list = superfast_uniq(full_symbol_list_with_dups)

# Step 4.c: Remove input constants from symbol list
for inputconst in input_constants:
    for symbol in full_symbol_list:
        if str(symbol) == str(inputconst):
            full_symbol_list.remove(symbol)

# Step 5.a: Convert each left-hand side to function notation
#   while separating and simplifying left- and right-hand sides
xx = sp.Symbol('xx')
func = []
for i in range(len(lr)):
    func.append(sp.sympify(sp.Function(lr[i].lhs)(xx)))

# Step 5.b: Mark each free variable as a function with argument xx
full_function_list = []
for symb in full_symbol_list:
    func = sp.sympify(sp.Function(str(symb))(xx))
    full_function_list.append(func)
    for i in range(len(rhss)):
        for var in rhss[i].free_symbols:
            if str(var) == str(symb):
                rhss[i] = rhss[i].subs(var,func)

# Step 6.a: Use SymPy's diff function to differentiate right-hand sides with respect to xx
#   and append "prm" notation to left-hand sides
lhss_deriv = []
rhss_deriv = []
for i in range(len(rhss)):
#        lhss_deriv.append(sp.sympify(str(lhss[i])+"prm"))
    lhss_deriv.append(sp.sympify(str(lhss[i])+"Prm"))
#        newrhs = sp.sympify(str(sp.diff(rhss[i],xx)).replace("(xx)","").replace(", xx","prm").replace("Derivative",""))
    newrhs = sp.sympify(str(sp.diff(rhss[i],xx)).replace("(xx)","").replace(", xx","Prm").replace("Derivative",""))
    rhss_deriv.append(newrhs)

# Step 7.b: Call the simplication function and then copy results
lhss_deriv_simp,rhss_deriv_simp = simplify_deriv(lhss_deriv,rhss_deriv)
lhss_deriv = lhss_deriv_simp
rhss_deriv = rhss_deriv_simp

# Step 8.b: Call the derivative function and populate dictionaries with the result
lhss_derivative = {}
rhss_derivative = {}
for index in range(len(dynamic_variables)):
    lhss_temp,rhss_temp = deriv_onevar(lhss_deriv,rhss_deriv,dynamic_variables,index)
    lhss_derivative[dynamic_variables[index]] = lhss_temp
    rhss_derivative[dynamic_variables[index]] = rhss_temp

# Step 9: Output original expression and each partial derivative expression in SymPy snytax
with open("second_partial_derivatives.txt", "w") as output:
    for i in range(len(lr)):
        right_side = lr[i].rhs
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                    "sp.pi").replace("sign(","sp.sign(").replace("Abs(",
                                                    "sp.Abs(").replace("Rational(","sp.Rational(")
        output.write(str(lr[i].lhs)+" = "+right_side_in_sp)
    for var in dynamic_variables:
        for i in range(len(lhss_derivative[var])):
            right_side = str(rhss_derivative[var][i])
            right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                        "sp.pi").replace("sign(","sp.sign(").replace("Abs(",
                                                        "sp.Abs(").replace("Rational(","sp.Rational(").replace("Prm",
                                                        "Prm_"+str(var))
            output.write(str(lhss_derivative[var][i]).replace("Prm","Prm_"+str(var))+" = "+right_side_in_sp+"\n")

In [68]:
# Define a function to return a set of reasonable input parameters
# This function contains three distinct sets of input parameters, and index differentiates between them
def reset_values():
    # Each variable corresponds to a specific set of input parameters
    # -f 20 -M 23 -m 10 -X 0.01 -Y 0.02 -Z -0.03 -x 0.04 -y -0.05 -z 0.06
#    values = {'m1': 2.300000000000000e+01, 'm2': 1.000000000000000e+01, 'eta': 2.112029384756657e-01, 'x': 2.129680593127755e+01, 'y': 0.000000000000000e+00, 'z': 0.000000000000000e+00, 'px': 0.000000000000000e+00, 'py': 2.335390295829903e-01, 'pz': 0.000000000000000e+00, 's1x': 5.290000000000000e+00, 's1y': 1.057982019428585e+01, 's1z': -1.587011986900481e+01, 's2x': 4.000000000000000e+00, 's2y': -4.999932020459767e+00, 's2z': 6.000056648964334e+00, 'KK': 5.288229332346335e-01, 'k0': -9.985821843899071e-01, 'k1': -8.345846602989615e-01, 'd1v2': -7.966696593617955e+01,'dheffSSv2':1.261873764525631e+01, 'tortoise': 1, 'EMgamma': 5.772156649015329e-01}
    # -f 20 -M 23 -m 17 -X 0.01 -Y 0.02 -Z -0.03 -x 0.04 -y -0.05 -z 0.06
#    values = {'m1': 2.300000000000000e+01, 'm2': 1.700000000000000e+01, 'eta': 2.443750000000000e-01, 'x': 1.874251805673713e+01, 'y': 0.000000000000000e+00, 'z': 0.000000000000000e+00, 'px': 0.000000000000000e+00, 'py': 2.515021172346453e-01, 'pz': 6.776263578034403e-21, 's1x': 5.290000000000000e+00, 's1y': 1.058297641042221e+01, 's1z': -1.586801532317281e+01, 's2x': 1.156000000000000e+01, 's2y': -1.445325205734002e+01, 's2z': 1.733728943540473e+01, 'KK': 4.015818741475341e-01, 'k0': -7.637538803933533e-01, 'k1': -6.532593752472639e-01, 'd1v2': -7.535873925781249e+01,'dheffSSv2':2.005903703125000e+01, 'tortoise': 1, 'EMgamma': 5.772156649015329e-01}
    # -f 20 -M 37 -m 7 -X 0.01 -Y 0.5 -Z -0.03 -x 0.04 -y -0.05 -z 0.06
    values = {'m1': 3.700000000000000e+01, 'm2': 7.000000000000000e+00, 'eta': 1.337809917355372e-01, 'x': 1.756956852666752e+01, 'y': 0.000000000000000e+00, 'z': 0.000000000000000e+00, 'px': 0.000000000000000e+00, 'py': 2.618958361721053e-01, 'pz': -4.336808689942018e-19, 's1x': 1.369000000000000e+01, 's1y': 6.840216429960809e+02, 's1z': -4.838787878117922e+01, 's2x': 1.960000000000000e+00, 's2y': -2.418418743988513e+00, 's2z': 2.966032835071963e+00, 'KK': 1.005859854813800e+00, 'k0': -1.876366249757123e+00, 'k1': -1.506733527559855e+00, 'd1v2': -8.434924523277951e+01,'dheffSSv2':2.367092569667376e+00, 'tortoise': 1, 'EMgamma': 5.772156649015329e-01}
    # Return the input values
    return values

import numpy as np# TYLERK
# Numerically evaluate right-hand sides using input values
def evaluate_expression(left_sides,right_sides,input_values):
    new_right_sides = []
    for i in range(len(right_sides)):
        term = sp.sympify(str(right_sides[i]).replace("(xx)",""))
        # Only look for the free variables in each expression to reduce computation time
        free_vars = term.free_symbols
        for variable in free_vars:
            term = term.subs(variable, input_values[str(variable)])
        # Evaluate each term to reduce computation time
        if sp.im(term.evalf())!=0:
            print(str(left_sides[i])+ " is complex! ")
        new_right_sides.append(sp.sympify(term.evalf()))
        # Store each subexpression in values numerically
        input_values[str(left_sides[i])] = new_right_sides[i]
    # Return the input values dictionary with all numerical right-hand added
    return input_values

# Create array of trusted LALSuite derivative values
# Note that position in the array corresponds to the index of the corresponding input values
# -f 20 -M 23 -m 10 -X 0.01 -Y 0.02 -Z -0.03 -x 0.04 -y -0.05 -z 0.06
#LALSuite_validated_values = {'d2Hdrdpphi': -9.094362869736959e-04, 'd2Hdr2': 8.112476173606075e-05}
# -f 20 -M 23 -m 17 -X 0.01 -Y 0.02 -Z -0.03 -x 0.04 -y -0.05 -z 0.06
#LALSuite_validated_values = {'d2Hdrdpphi': -1.242455937978697e-03, 'd2Hdr2': 1.147405161494248e-04}
# -f 20 -M 37 -m 7 -X 0.01 -Y 0.5 -Z -0.03 -x 0.04 -y -0.05 -z 0.06
LALSuite_validated_values = {'d2Hdrdpphi': -1.450075972561637e-03, 'd2Hdr2': 1.345305858572373e-04}

In [69]:
values = reset_values()
values = evaluate_expression(lhss,rhss,values)
withrespectto_x  = evaluate_expression(lhss_derivative[x],rhss_derivative[x],values)
withrespectto_y  = evaluate_expression(lhss_derivative[y],rhss_derivative[y],values)
withrespectto_z  = evaluate_expression(lhss_derivative[z],rhss_derivative[z],values)
withrespectto_px = evaluate_expression(lhss_derivative[px],rhss_derivative[px],values)
withrespectto_py = evaluate_expression(lhss_derivative[py],rhss_derivative[py],values)
withrespectto_pz = evaluate_expression(lhss_derivative[pz],rhss_derivative[pz],values)

r = values['r']
pphi = r*values['py']
ptheta = -r*values['pz']

d2Hdx2 = withrespectto_x['Hrealprm_xPrm']/values['eta']
d2Hdxdpy = withrespectto_py['Hrealprm_xPrm']/values['eta']
d2Hdxdpz = withrespectto_pz['Hrealprm_xPrm']/values['eta']

dpydr = -pphi/r/r
dpzdr = ptheta/r/r

d2Hdpydx = withrespectto_x['Hrealprm_pyPrm']/values['eta']
d2Hdpydy = withrespectto_y['Hrealprm_pyPrm']/values['eta']
d2Hdpydz = withrespectto_z['Hrealprm_pyPrm']/values['eta']
d2Hdpydpx = withrespectto_px['Hrealprm_pyPrm']/values['eta']
d2Hdpydpy = withrespectto_py['Hrealprm_pyPrm']/values['eta']
d2Hdpydpz = withrespectto_pz['Hrealprm_pyPrm']/values['eta']

d2Hdpzdx = withrespectto_x['Hrealprm_pzPrm']/values['eta']
d2Hdpzdy = withrespectto_y['Hrealprm_pzPrm']/values['eta']
d2Hdpzdz = withrespectto_z['Hrealprm_pzPrm']/values['eta']
d2Hdpzdpx = withrespectto_px['Hrealprm_pzPrm']/values['eta']
d2Hdpzdpy = withrespectto_py['Hrealprm_pzPrm']/values['eta']
d2Hdpzdpz = withrespectto_pz['Hrealprm_pzPrm']/values['eta']

dHdpy = withrespectto_py['Hrealprm_py']/values['eta']
dHdpz = withrespectto_pz['Hrealprm_pz']/values['eta']

dpphidr = values['py']
dpthetadr = -values['pz']

logTerms is complex! 
deltaU is complex! 
deltaT is complex! 
deltaUUSCOREu is complex! 
deltaTUSCOREr is complex! 
Lamb is complex! 
rho2xi2Lamb is complex! 
invrho2xi2Lamb is complex! 
invLamb is complex! 
invLambsq is complex! 
rho2invLamb is complex! 
expnu is complex! 
expMUexpnu is complex! 
expnusq is complex! 
expMUsqexpnusq is complex! 
invexpnuexpMU is complex! 
invexpMU is complex! 
invexpMUsq is complex! 
expnuinvexpMU2 is complex! 
invexpMUcubinvexpnu is complex! 
deltaR is complex! 
B is complex! 
sqrtdeltaT is complex! 
sqrtdeltaR is complex! 
deltaTsqrtdeltaR is complex! 
sqrtdeltaTdeltaTsqrtdeltaR is complex! 
invdeltaTsqrtdeltaTsqrtdeltaR is complex! 
invdeltaT is complex! 
invsqrtdeltaT is complex! 
invsqrtdeltaR is complex! 
w is complex! 
LambUSCOREr is complex! 
BR is complex! 
wr is complex! 
nurpt2 is complex! 
nurpt1 is complex! 
nur is complex! 
mur is complex! 
wcospt2 is complex! 
wcospt1 is complex! 
wcos is complex! 
nucospt3 is complex! 
nucospt2 is compl

LambUSCORErprm_y is complex! 
wrprm_y is complex! 
nurprm_y is complex! 
wcospt1prm_y is complex! 
wcosprm_y is complex! 
nucospt3prm_y is complex! 
nucospt2prm_y is complex! 
nucospt1prm_y is complex! 
nucosprm_y is complex! 
prTtimesoneminuscsi1invprm_y is complex! 
tmppxprm_y is complex! 
pnprm_y is complex! 
prprm_y is complex! 
Hnspt7prm_y is complex! 
Hnspt6prm_y is complex! 
Hnspt4prm_y is complex! 
Hnspt3prm_y is complex! 
Hnspt2prm_y is complex! 
Hnsprm_y is complex! 
Qpt3prm_y is complex! 
Qpt2prm_y is complex! 
Qprm_y is complex! 
ppprm_y is complex! 
sKerrmultfactprm_y is complex! 
sStarmultfactprm_y is complex! 
deltaSigmaStarUSCOREx1prm_y is complex! 
deltaSigmaStarUSCOREy1prm_y is complex! 
deltaSigmaStarUSCOREz1prm_y is complex! 
pp2prm_y is complex! 
ppu2prm_y is complex! 
sMultiplier1pt6prm_y is complex! 
sMultiplier1pt5prm_y is complex! 
sMultiplier1pt4prm_y is complex! 
sMultiplier1pt3prm_y is complex! 
sMultiplier1pt2prm_y is complex! 
sMultiplier1pt1prm_y is compl

pp2prm_py is complex! 
ppu2prm_py is complex! 
sMultiplier1pt6prm_py is complex! 
sMultiplier1pt5prm_py is complex! 
sMultiplier1pt4prm_py is complex! 
sMultiplier1pt3prm_py is complex! 
sMultiplier1pt2prm_py is complex! 
sMultiplier1pt1prm_py is complex! 
sMultiplier1prm_py is complex! 
sMultiplier2pt5prm_py is complex! 
sMultiplier2pt4prm_py is complex! 
sMultiplier2pt3prm_py is complex! 
sMultiplier2pt2prm_py is complex! 
sMultiplier2pt1prm_py is complex! 
sMultiplier2prm_py is complex! 
deltaSigmaStarUSCOREx2prm_py is complex! 
deltaSigmaStarUSCOREy2prm_py is complex! 
deltaSigmaStarUSCOREz2prm_py is complex! 
deltaSigmaStarUSCORExprm_py is complex! 
deltaSigmaStarUSCOREyprm_py is complex! 
deltaSigmaStarUSCOREzprm_py is complex! 
sxprm_py is complex! 
syprm_py is complex! 
szprm_py is complex! 
sxiprm_py is complex! 
svprm_py is complex! 
snprm_py is complex! 
s3prm_py is complex! 
sqrtQprm_py is complex! 
oneplus2sqrtQprm_py is complex! 
oneplus1sqrtQprm_py is complex! 
twoB1psqr

Hwrpt4Prm is complex! 
Hwrpt3cPrm is complex! 
Hwrpt3bPrm is complex! 
Hwrpt3aPrm is complex! 
Hwrpt3Prm is complex! 
Hwrpt2gPrm is complex! 
Hwrpt2fPrm is complex! 
Hwrpt2ePrm is complex! 
Hwrpt2bPrm is complex! 
Hwrpt2aPrm is complex! 
Hwrpt2Prm is complex! 
Hwrpt1bPrm is complex! 
Hwrpt1aPrm is complex! 
Hwrpt1Prm is complex! 
HwrPrm is complex! 
Hwcospt9Prm is complex! 
Hwcospt8Prm is complex! 
Hwcospt7Prm is complex! 
Hwcospt6Prm is complex! 
Hwcospt5Prm is complex! 
Hwcospt3Prm is complex! 
Hwcospt2Prm is complex! 
Hwcospt1Prm is complex! 
HwcosPrm is complex! 
deltaTsqrtQPrm is complex! 
invdeltatTsqrtQPrm is complex! 
HSOLpt5Prm is complex! 
HSOLpt4Prm is complex! 
HSOLpt3Prm is complex! 
HSOLpt2Prm is complex! 
HSOLpt1Prm is complex! 
HSOLPrm is complex! 
deltaTsqrtQplusQPrm is complex! 
invdeltaTsqrtQplusQPrm is complex! 
HSONLmult2Prm is complex! 
HSONLmultPrm is complex! 
HSONLpt1aPrm is complex! 
HSONLpt1Prm is complex! 
HSONLpt2dPrm is complex! 
HSONLpt2cPrm is complex! 


snprm_yPrm is complex! 
s3prm_yPrm is complex! 
sqrtQprm_yPrm is complex! 
oneplus2sqrtQprm_yPrm is complex! 
oneplus1sqrtQprm_yPrm is complex! 
twoB1psqrtQsqrtQprm_yPrm is complex! 
invtwoB1psqrtQsqrtQprm_yPrm is complex! 
expMUsqsqrtQplusQprm_yPrm is complex! 
Hwrpt4aprm_yPrm is complex! 
Hwrpt4prm_yPrm is complex! 
Hwrpt3cprm_yPrm is complex! 
Hwrpt3bprm_yPrm is complex! 
Hwrpt3aprm_yPrm is complex! 
Hwrpt3prm_yPrm is complex! 
Hwrpt2gprm_yPrm is complex! 
Hwrpt2fprm_yPrm is complex! 
Hwrpt2eprm_yPrm is complex! 
Hwrpt2cprm_yPrm is complex! 
Hwrpt2bprm_yPrm is complex! 
Hwrpt2aprm_yPrm is complex! 
Hwrpt2prm_yPrm is complex! 
Hwrpt1bprm_yPrm is complex! 
Hwrpt1aprm_yPrm is complex! 
Hwrpt1prm_yPrm is complex! 
Hwrprm_yPrm is complex! 
Hwcospt9prm_yPrm is complex! 
Hwcospt8prm_yPrm is complex! 
Hwcospt7prm_yPrm is complex! 
Hwcospt6prm_yPrm is complex! 
Hwcospt5prm_yPrm is complex! 
Hwcospt4prm_yPrm is complex! 
Hwcospt3prm_yPrm is complex! 
Hwcospt2prm_yPrm is complex! 
Hwcospt1prm_

...

svprm_yPrm is complex! 
snprm_yPrm is complex! 
s3prm_yPrm is complex! 
sqrtQprm_yPrm is complex! 
oneplus2sqrtQprm_yPrm is complex! 
oneplus1sqrtQprm_yPrm is complex! 
twoB1psqrtQsqrtQprm_yPrm is complex! 
invtwoB1psqrtQsqrtQprm_yPrm is complex! 
expMUsqsqrtQplusQprm_yPrm is complex! 
Hwrpt4aprm_yPrm is complex! 
Hwrpt4prm_yPrm is complex! 
Hwrpt3cprm_yPrm is complex! 
Hwrpt3bprm_yPrm is complex! 
Hwrpt3aprm_yPrm is complex! 
Hwrpt3prm_yPrm is complex! 
Hwrpt2gprm_yPrm is complex! 
Hwrpt2fprm_yPrm is complex! 
Hwrpt2eprm_yPrm is complex! 
Hwrpt2cprm_yPrm is complex! 
Hwrpt2bprm_yPrm is complex! 
Hwrpt2aprm_yPrm is complex! 
Hwrpt2prm_yPrm is complex! 
Hwrpt1bprm_yPrm is complex! 
Hwrpt1aprm_yPrm is complex! 
Hwrpt1prm_yPrm is complex! 
Hwrprm_yPrm is complex! 
Hwcospt9prm_yPrm is complex! 
Hwcospt8prm_yPrm is complex! 
Hwcospt7prm_yPrm is complex! 
Hwcospt6prm_yPrm is complex! 
Hwcospt5prm_yPrm is complex! 
Hwcospt4prm_yPrm is complex! 
Hwcospt3prm_yPrm is complex! 
Hwcospt2prm_yPrm i

HSONLpt1prm_pyPrm is complex! 
HSONLpt2cprm_pyPrm is complex! 
HSONLpt2bprm_pyPrm is complex! 
HSONLpt2aprm_pyPrm is complex! 
HSONLpt2prm_pyPrm is complex! 
HSONLpt3cprm_pyPrm is complex! 
HSONLpt3bprm_pyPrm is complex! 
HSONLpt3aprm_pyPrm is complex! 
HSONLpt3prm_pyPrm is complex! 
HSONLpt4eprm_pyPrm is complex! 
HSONLpt4dprm_pyPrm is complex! 
HSONLpt4cprm_pyPrm is complex! 
HSONLpt4bprm_pyPrm is complex! 
HSONLpt4aprm_pyPrm is complex! 
HSONLpt4prm_pyPrm is complex! 
HSONLprm_pyPrm is complex! 
Hsprm_pyPrm is complex! 
Hsspt1prm_pyPrm is complex! 
Hssprm_pyPrm is complex! 
Hprm_pyPrm is complex! 
Hrealprm_pyPrm is complex! 
Hnspt4prm_pzPrm is complex! 
Hnspt3prm_pzPrm is complex! 
Hnspt2prm_pzPrm is complex! 
Hnsprm_pzPrm is complex! 
Qprm_pzPrm is complex! 
ppprm_pzPrm is complex! 
sKerrmultfactprm_pzPrm is complex! 
sStarmultfactprm_pzPrm is complex! 
deltaSigmaStarUSCOREx1prm_pzPrm is complex! 
deltaSigmaStarUSCOREy1prm_pzPrm is complex! 
deltaSigmaStarUSCOREz1prm_pzPrm is compl

Hwrpt2eprm_xPrm is complex! 
Hwrpt2bprm_xPrm is complex! 
Hwrpt2aprm_xPrm is complex! 
Hwrpt2prm_xPrm is complex! 
Hwrpt1bprm_xPrm is complex! 
Hwrpt1aprm_xPrm is complex! 
Hwrpt1prm_xPrm is complex! 
Hwrprm_xPrm is complex! 
Hwcospt9prm_xPrm is complex! 
Hwcospt8prm_xPrm is complex! 
Hwcospt7prm_xPrm is complex! 
Hwcospt6prm_xPrm is complex! 
Hwcospt5prm_xPrm is complex! 
Hwcospt3prm_xPrm is complex! 
Hwcospt2prm_xPrm is complex! 
Hwcospt1prm_xPrm is complex! 
Hwcosprm_xPrm is complex! 
deltaTsqrtQprm_xPrm is complex! 
invdeltatTsqrtQprm_xPrm is complex! 
HSOLpt5prm_xPrm is complex! 
HSOLpt4prm_xPrm is complex! 
HSOLpt3prm_xPrm is complex! 
HSOLpt2prm_xPrm is complex! 
HSOLpt1prm_xPrm is complex! 
HSOLprm_xPrm is complex! 
deltaTsqrtQplusQprm_xPrm is complex! 
invdeltaTsqrtQplusQprm_xPrm is complex! 
HSONLmult2prm_xPrm is complex! 
HSONLmultprm_xPrm is complex! 
HSONLpt1aprm_xPrm is complex! 
HSONLpt1prm_xPrm is complex! 
HSONLpt2dprm_xPrm is complex! 
HSONLpt2cprm_xPrm is complex! 
H

Hsspt1prm_zPrm is complex! 
Hssprm_zPrm is complex! 
Hprm_zPrm is complex! 
Hrealprm_zPrm is complex! 
Hwrpt2cprm_pxPrm is complex! 
Hwrpt2aprm_pxPrm is complex! 
Hwrpt2prm_pxPrm is complex! 
Hwrprm_pxPrm is complex! 
Hwcospt4prm_pxPrm is complex! 
Hwcospt2prm_pxPrm is complex! 
Hwcospt1prm_pxPrm is complex! 
Hwcosprm_pxPrm is complex! 
HSONLpt1prm_pxPrm is complex! 
HSONLpt2prm_pxPrm is complex! 
HSONLpt3prm_pxPrm is complex! 
HSONLpt4prm_pxPrm is complex! 
HSONLprm_pxPrm is complex! 
Hsprm_pxPrm is complex! 
Hprm_pxPrm is complex! 
Hrealprm_pxPrm is complex! 
Hnspt4prm_pyPrm is complex! 
Hnspt3prm_pyPrm is complex! 
Hnspt2prm_pyPrm is complex! 
Hnsprm_pyPrm is complex! 
Qprm_pyPrm is complex! 
ppprm_pyPrm is complex! 
sKerrmultfactprm_pyPrm is complex! 
sStarmultfactprm_pyPrm is complex! 
deltaSigmaStarUSCOREx1prm_pyPrm is complex! 
deltaSigmaStarUSCOREy1prm_pyPrm is complex! 
deltaSigmaStarUSCOREz1prm_pyPrm is complex! 
pp2prm_pyPrm is complex! 
ppu2prm_pyPrm is complex! 
sMultiplie

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [67]:
new_d2Hdr2 = (d2Hdx2 + d2Hdxdpy*dpydr + d2Hdxdpz*dpzdr
              - (d2Hdpydx + d2Hdpydpy*dpydr + d2Hdpydpz*dpzdr)*pphi/r/r - dHdpy*(dpphidr/r/r - pphi*2/r/r/r)
              + (d2Hdpzdx + d2Hdpzdpy*dpydr + d2Hdpzdpz*dpzdr)*ptheta/r/r + dHdpz*(dpthetadr/r/r - ptheta*2/r/r/r)
             )

print(new_d2Hdr2)
print(np.iscomplex(new_d2Hdr2))

tru_d2Hdr2 = LALSuite_validated_values['d2Hdr2']
print("new d2Hdr2 = %.15e" % new_d2Hdr2)
print("tru d2Hdr2 = %.15e" % tru_d2Hdr2)
print("relative diff = %.15e" % ((tru_d2Hdr2-new_d2Hdr2)/tru_d2Hdr2))

0.00102946193609586 + 0.000152300041307563*I
False


TypeError: can't convert complex to float

In [5]:
values = reset_values()
values = evaluate_expression(lhss,rhss,values)
withrespectto_x  = evaluate_expression(lhss_derivative[x],rhss_derivative[x],values)
withrespectto_py = evaluate_expression(lhss_derivative[py],rhss_derivative[py],values)
withrespectto_pz = evaluate_expression(lhss_derivative[pz],rhss_derivative[pz],values)

r = values['r']
pphi = r*values['py']
ptheta = -r*values['pz']

d2Hdxdx = withrespectto_x['Hrealprm_xPrm']/values['eta']
d2Hdpydx = withrespectto_x['Hrealprm_pyPrm']/values['eta']
d2Hdpzdx = withrespectto_x['Hrealprm_pzPrm']/values['eta']
d2Hdxdpy = withrespectto_py['Hrealprm_xPrm']/values['eta']
d2Hdpydpy = withrespectto_py['Hrealprm_pyPrm']/values['eta']
d2Hdpzdpy = withrespectto_py['Hrealprm_pzPrm']/values['eta']

dxdpphi = 1./values['py']
dpydpphi = -1./r
drdpphi = 1./values['py']

dHdpy = withrespectto_py['Hrealprm_py']/values['eta']
dHdpz = withrespectto_pz['Hrealprm_pz']/values['eta']

In [6]:
new_d2Hdrdpphi = (#d2Hdxdx/dxdpphi + d2Hdxdpy/dpydpphi
                  - (d2Hdpydx*dxdpphi + d2Hdpydpy*dpydpphi)*pphi/r/r# - dHdpy*pphi*2/r/r/r*drdpphi
                  + (d2Hdpzdx*dxdpphi + d2Hdpzdpy*dpydpphi)*ptheta/r/r# - dHdpz*ptheta*2/r/r/r*drdpphi
                 )

new_d2Hdrdpphi = (#d2Hdxdx/dxdpphi + d2Hdxdpy/dpydpphi
                  - (d2Hdpydpy*dpydpphi)*pphi/r/r - dHdpy*pphi*2/r/r/r*drdpphi
                  + (d2Hdpzdpy*dpydpphi)*ptheta/r/r - dHdpz*ptheta*2/r/r/r*drdpphi
                 )

#new_d2Hdrdpphi /= values['eta']

tru_d2Hdrdpphi = LALSuite_validated_values['d2Hdrdpphi']

print("new d2Hdrdpphi = %.15e" % new_d2Hdrdpphi)
print("tru d2Hdrdpphi = %.15e" % tru_d2Hdrdpphi)
print("relative diff = %.15e" % ((tru_d2Hdrdpphi-new_d2Hdrdpphi)/tru_d2Hdrdpphi))

new d2Hdrdpphi = -1.237998396435706e-03
tru d2Hdrdpphi = -1.242455937978697e-03
relative diff = 3.587685813826620e-03


In [7]:
dHdx = rhss_derivative[x][-1]
print(dHdx)
#d2Hdr2 = dHdx - dHdpy*pphi/r/r + dHdpz*ptheta/r/r

-HPrm*Hprm_pz*eta**2/(2*eta*(H - 1) + 1)**(3/2) + Hprm_pzPrm*eta/sqrt(2*eta*(H - 1) + 1)
